In [1]:

import gradio as gr
import torch
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from detoxify import Detoxify

# Initialize AI Models
sia = SentimentIntensityAnalyzer()
sentiment_pipeline = pipeline("sentiment-analysis")

# Function to process conversation and detect risk level
def chatbot_response(message, history):
    """
    AI-powered chatbot that:
    - Greets the user
    - Analyzes messages using AI models for suicide risk detection
    - Provides emergency contacts for high-risk messages
    """

    # Step 1: Greeting & Friendly Response
    message = message.strip().lower()
    if "hello" in message or "hi" in message:
        return "👋 Hello, my friend! How are you feeling today? 💙"

    if "help" in message:
        return "I'm here for you! What do you need help with? 😊"

    # Step 2: Analyze Message for Sentiment & Toxicity
    sentiment_score = sia.polarity_scores(message)["compound"]
    toxicity_scores = Detoxify("original").predict(message)

    # Extract AI Model Predictions
    toxicity = toxicity_scores["toxicity"]
    severe_toxicity = toxicity_scores["severe_toxicity"]
    insult = toxicity_scores["insult"]
    threat = toxicity_scores["threat"]

    # Step 3: Advanced Negative Sentiment Scoring Using DistilBERT
    bert_sentiment = sentiment_pipeline(message)
    negativity_score = bert_sentiment[0]["score"] if bert_sentiment[0]["label"] == "NEGATIVE" else 0

    # Step 4: Automatically Classify Suicide Risk Level
    if toxicity > 0.7 or severe_toxicity > 0.5 or threat > 0.5 or sentiment_score < -0.7 or negativity_score > 0.7:
        return f"🚨 **High Risk Detected!**\nI'm really concerned about you. Please seek immediate help! 💙\n📞 **Emergency Hotline:** Call 988 (USA) / 116 123 (UK & EU) / 113 (Netherlands)\n\n💡 **Negative Score:** {negativity_score:.2f}"
    elif toxicity > 0.4 or insult > 0.4 or sentiment_score < -0.4 or negativity_score > 0.4:
        return f"⚠️ **Moderate Risk.**\nIt sounds like you're going through something difficult. Would you like to talk more? I'm here to listen. 💙\n\n💡 **Negative Score:** {negativity_score:.2f}"
    else:
        return f"✅ **No Risk Detected.**\nI'm always here if you need someone to talk to. 😊 Just tell me what's happening.\n\n💡 **Negative Score:** {negativity_score:.2f}"

# Gradio Chat Interface
iface = gr.ChatInterface(
    fn=chatbot_response, 
    title="Vera",
    description="A chatbot that uses AI models to detect suicide risk and provide support. 💙",
    theme="compact"
)

# Launch the chatbot
iface.launch()

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0
/opt/anaconda3/lib/python3.12/site-packages/gradio/blocks.py:1057: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-679f4e04-1e1c80d34bd060532062a319;3215f815-5fa6-48d8-99a3-0034bbc8ec3c)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/opt/anaconda3/lib/python3.12/site-packages/gradio/components/chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and '

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [2]:
import gradio as gr
import torch
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from detoxify import Detoxify

# Initialize AI Models
sia = SentimentIntensityAnalyzer()
sentiment_pipeline = pipeline("sentiment-analysis")

# Conversation Memory
conversation_history = {}

# Function to process conversation and detect risk level
def vera_chat(message, history):
    """
    AI-powered chatbot that:
    - Responds relevantly to each message
    - Listens and provides emotional support
    - Uses AI to detect suicide risk
    - Encourages self-care for moderate distress
    - Urges immediate action for high-risk messages
    """

    user_id = "user_1"  # For future session memory
    message = message.strip().lower()

    # Step 1: Start a Friendly Conversation
    if message in ["hello", "hi", "hey", "hola", "bonjour"]:
        conversation_history[user_id] = []  # Reset chat history
        return "👋 Hi! I'm Vera. How are you feeling today? 💙"

    if "how are you" in message:
        return "😊 I'm just a chatbot, but I'm happy to be here for you. What’s on your mind?"

    if "help" in message:
        return "💙 Of course! Tell me what’s been going on. I’m here to listen."

    # Step 2: Analyze Sentiment & Toxicity
    sentiment_score = sia.polarity_scores(message)["compound"]
    toxicity_scores = Detoxify("original").predict(message)

    # Extract AI Model Predictions
    toxicity = toxicity_scores["toxicity"]
    severe_toxicity = toxicity_scores["severe_toxicity"]
    insult = toxicity_scores["insult"]
    threat = toxicity_scores["threat"]

    # Advanced BERT Sentiment Scoring
    bert_sentiment = sentiment_pipeline(message)
    negativity_score = bert_sentiment[0]["score"] if bert_sentiment[0]["label"] == "NEGATIVE" else 0

    # Store message history
    conversation_history[user_id].append(message)

    # Step 3: Handle Suicide Risk Detection
    if toxicity > 0.7 or severe_toxicity > 0.5 or threat > 0.5 or sentiment_score < -0.7 or negativity_score > 0.7:
        return ("🚨 **I’m really concerned about you. Please don’t go through this alone.**\n"
                "💙 **You are important, and you deserve support.**\n"
                "📞 **Emergency Hotline:** Call 988 (USA) / 116 123 (UK & EU) / 113 (Netherlands)\n"
                "🫂 **Please don’t do anything right now. Breathe. Stay with me.**\n"
                "Would you like to talk about what’s been making you feel this way?")

    # Step 4: If User is Sad, Respond with Empathy & Ask More
    if negativity_score > 0.4 and toxicity < 0.5:
        return "💙 That sounds really tough. What’s been on your mind lately? I’m here for you."

    # Step 5: If User Expresses Stress or Anxiety, Suggest Self-Care
    if "stressed" in message or "anxious" in message or "overwhelmed" in message:
        return "🌿 I understand. When I feel overwhelmed, I try deep breathing or taking a short walk. Have you tried that before?"

    if "tired" in message:
        return "😴 I hear you. Sometimes rest is the best thing. Have you been getting enough sleep lately?"

    if "alone" in message:
        return "💙 I want you to know that you’re not alone. I’m here to talk, and I care about you."

    # Step 6: Default Friendly Response if No Specific Issue Detected
    return "💙 I’m always here to listen. What’s been on your mind today?"

# Gradio Chat Interface
iface = gr.ChatInterface(
    fn=vera_chat, 
    title="Vera - Your AI Support Friend 💙",
    description="A chatbot that listens, supports, and helps when you need it most. 💙",
    theme="compact"
)

# Launch Vera
iface.launch()


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0
/opt/anaconda3/lib/python3.12/site-packages/gradio/blocks.py:1057: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-679f4e05-155f6e4a0afec9da584de0b8;7d700592-7331-42bf-ba33-65eef51521d1)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/opt/anaconda3/lib/python3.12/site-packages/gradio/components/chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and '

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
